In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, preprocessing
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from catboost import Pool, CatBoostClassifier
import lightgbm as lgb
import seaborn as sns
#from itertools import izip
from impyute.imputation.cs import mice


/Users/s0c02nj/anaconda2/envs/graph/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
#Importing
train_data = pd.read_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/training_v2.csv')
test_data =  pd.read_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/unlabeled.csv')

sub_data =  pd.read_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/samplesubmission.csv')
data_dict = pd.read_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/WiDS Datathon 2020 Dictionary.csv')

In [ ]:
#train_data.groupby('hospital_death').hist()
#list_plot = list(np.arange(169,173))

In [ ]:
# df = train_data.iloc[:,list_plot]
# df['hospital_death'] = train_data['hospital_death']

In [ ]:
#df.groupby('hospital_death').hist()
#train_data[['height','hospital_death']].groupby('hospital_death').hist()

In [3]:
def weighted_classt(x): 
    if pd.isna(x):
        return np.nan
    elif x < 15: 
        return 'very severely underweight' 
    elif x >= 15 and x < 16: 
        return 'severely weight' 
    elif x >=16 and x < 18.5: 
        return 'underweight' 
    elif x >= 18.5 and x < 25: 
        return 'healthy weight' 
    elif x >= 25 and x < 30: 
        return 'overweight'
    elif x >= 30 and x < 35: 
        return 'class 1' 
    elif x >= 35 and x < 40: 
        return 'class 2' 
    else: 
        return 'class 3' 

In [4]:
train_data['weightclass'] = train_data['bmi'].map(weighted_classt)
test_data['weightclass'] = test_data['bmi'].map(weighted_classt)

In [5]:
cat_var = list(data_dict[data_dict['Data Type'].isin(['binary','string','integer'])]['Variable Name'])
cat_var.remove('icu_admit_type')
cat_var.remove('bmi')
cat_var = cat_var[1:]

#to_label_encode = list(data_dict[data_dict['Data Type'].isin(['string'])]['Variable Name'])

In [6]:
cat_var = cat_var + ['weightclass']

In [7]:
cont_var = list(data_dict[data_dict['Data Type'].isin(['numeric'])]['Variable Name'])
cont_var.remove('pred')
cont_var = cont_var + ['bmi']

#### Baseline

In [8]:
x_train = train_data.drop('hospital_death',axis=1)
y_train = train_data['hospital_death']

In [9]:
x_test = test_data.drop('hospital_death',axis=1)

In [10]:
x_comb = pd.concat([x_train,x_test],sort=False)

In [11]:
list_drop = ['encounter_id', 'patient_id', 'readmission_status']

In [12]:
# # Remove Features with more than 70 percent missing values
#x_comb = x_comb.drop(['readmission_status'],axis = 1)
x_comb = x_comb.drop(list_drop,axis = 1)

In [13]:
new_cat_var = list(set(cat_var).intersection(set(x_comb.columns)))

In [14]:
for col in tqdm(new_cat_var):
    le = LabelEncoder()
    x_comb[col] = le.fit_transform(x_comb[col].astype(str))


100%|██████████| 30/30 [00:03<00:00,  8.77it/s]


In [15]:
x_comb["apache_4a_hospital_death_prob"] = x_comb["apache_4a_hospital_death_prob"].replace(-1, np.nan)
x_comb["apache_4a_icu_death_prob"] =  x_comb["apache_4a_icu_death_prob"].replace(-1, np.nan)

In [16]:
age_cat = []

for i in tqdm(range(0,len(x_comb))):
    
    val = x_comb['age'].iloc[i]
    
    if val >= 15 and val <= 24: 
        age_cat.append('igen')
    
    elif val >= 25 and val <= 54: 
       age_cat.append('Prime_working_Age')
    
    elif val >= 55 and val <= 64: 
        age_cat.append('Mature_working_Age')
        
    else: 
        age_cat.append('Elderly_working_Age')


100%|██████████| 131021/131021 [00:01<00:00, 70998.77it/s]


In [17]:
x_comb['age_category'] = age_cat
le = LabelEncoder()
x_comb['age_category'] = le.fit_transform(x_comb['age_category'].astype(str))

In [18]:
x_comb1 = x_comb.copy()

In [ ]:
# def apache_prob(x): 
#     if pd.isna(x):
#         return np.nan
#     else:
#         return str(x)

In [ ]:
# x_comb1['apache_4a_hospital_death_prob'] = x_comb1['apache_4a_hospital_death_prob'].apply(lambda x: 
#                                                                                           apache_prob(x))

In [ ]:
#x_comb1['apache_4a_hospital_death_prob'] = x_comb1['apache_4a_hospital_death_prob'].astype(str)

In [19]:
list_col = list(x_comb1.columns)

list_col = list(filter(lambda x: x!= 'apache_4a_hospital_death_prob', list_col))

In [21]:
# for col in tqdm(list_col):
#     print (col)
#     le = LabelEncoder()
#     x_comb1[col] = le.fit_transform(x_comb1[col].astype(str))

In [22]:
import datawig


In [23]:
#Initialize a SimpleImputer model
imputer = datawig.SimpleImputer(
    input_columns = list_col, # column(s) containing information about the column we want to impute
    output_column= 'apache_4a_hospital_death_prob', # the column we'd like to impute values for
    #output_path = 'imputer_model' # stores model data and metrics
    )

In [24]:
#Fit an imputer model on the train data
imputer.fit(train_df=x_comb1, num_epochs = 50)


2020-02-23 03:09:20,382 [INFO]  
========== start: fit model
2020-02-23 03:09:20,382 [WARNING]  Already bound, ignoring bind()
2020-02-23 03:09:23,302 [INFO]  Epoch[0] Batch [0-3685]	Speed: 20224.64 samples/sec	cross-entropy=1.318600	apache_4a_hospital_death_prob-accuracy=0.000000
2020-02-23 03:09:26,376 [INFO]  Epoch[0] Train-cross-entropy=1.070701
2020-02-23 03:09:26,377 [INFO]  Epoch[0] Train-apache_4a_hospital_death_prob-accuracy=0.000000
2020-02-23 03:09:26,378 [INFO]  Epoch[0] Time cost=5.993
2020-02-23 03:09:26,382 [INFO]  Saved checkpoint to "apache_4a_hospital_death_prob/model-0000.params"
2020-02-23 03:09:26,615 [INFO]  Epoch[0] Validation-cross-entropy=0.446889
2020-02-23 03:09:26,616 [INFO]  Epoch[0] Validation-apache_4a_hospital_death_prob-accuracy=0.000000
2020-02-23 03:09:30,248 [INFO]  Epoch[1] Batch [0-3685]	Speed: 16242.84 samples/sec	cross-entropy=0.709223	apache_4a_hospital_death_prob-accuracy=0.000000
2020-02-23 03:09:34,006 [INFO]  Epoch[1] Train-cross-entropy=0.6

2020-02-23 03:10:55,151 [INFO]  Epoch[12] Train-apache_4a_hospital_death_prob-accuracy=0.000000
2020-02-23 03:10:55,152 [INFO]  Epoch[12] Time cost=9.042
2020-02-23 03:10:55,156 [INFO]  Saved checkpoint to "apache_4a_hospital_death_prob/model-0012.params"
2020-02-23 03:10:55,421 [INFO]  Epoch[12] Validation-cross-entropy=0.330660
2020-02-23 03:10:55,422 [INFO]  Epoch[12] Validation-apache_4a_hospital_death_prob-accuracy=0.000000
2020-02-23 03:10:59,803 [INFO]  Epoch[13] Batch [0-3685]	Speed: 13468.07 samples/sec	cross-entropy=0.594132	apache_4a_hospital_death_prob-accuracy=0.000000
2020-02-23 03:11:03,968 [INFO]  Epoch[13] Train-cross-entropy=0.553126
2020-02-23 03:11:03,969 [INFO]  Epoch[13] Train-apache_4a_hospital_death_prob-accuracy=0.000000
2020-02-23 03:11:03,970 [INFO]  Epoch[13] Time cost=8.547
2020-02-23 03:11:03,975 [INFO]  Saved checkpoint to "apache_4a_hospital_death_prob/model-0013.params"
2020-02-23 03:11:04,193 [INFO]  Epoch[13] Validation-cross-entropy=0.342560
2020-02-

In [25]:
#Impute missing values and return original dataframe with predictions
imputed = imputer.predict(x_comb1)

/Users/s0c02nj/anaconda2/envs/graph/lib/python3.6/site-packages/datawig/imputer.py:822: FutureWarning: The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'
  if data_frame.columns.contains(imputation_col):


In [26]:
x_comb['apache_4a_hospital_death_prob'] = imputed['apache_4a_hospital_death_prob_imputed']

In [ ]:
#'gender, age, height, weight'

In [31]:
#Initialize a SimpleImputer model
imputer = datawig.SimpleImputer(
    input_columns = ['age','gender','weight','height'],
    output_column = 'bmi'
    )

In [ ]:
#Impute missing values and return original dataframe with predictions
imputed = imputer.predict(x_comb1)

In [ ]:
count_var = []
for i,col in tqdm(enumerate(new_cat_var)):
    counter = Counter(x_comb[col])
    x_comb[str(col)+'count'] = x_comb[col].apply(lambda x:counter[x])
    count_var.append(str(col)+'count')

In [ ]:
#List to be biined -----> VITALS
#to_binned_columns = list(data_dict[data_dict['Category'] == 'labs blood gas']['Variable Name'].unique())
to_binned_columns = list(data_dict[data_dict['Category'] == 'labs']['Variable Name'].unique())
to_binned_columns = list(set(to_binned_columns).intersection(set(x_comb.columns)))

In [ ]:
binned_cols = []

for cols in tqdm(to_binned_columns) :
    x_comb[cols+'_binned'] = list(pd.cut(x_comb[cols], 10 ,labels=False))
    binned_cols.append(cols+'_binned')
    

In [ ]:
x_comb['height'+'_binned'] = list(pd.cut(x_comb['height'], 8,labels=False))

In [ ]:
featured_cols = binned_cols + ['age_category','height'+'_binned']

In [ ]:
#2way interaction
bias_cols = ['gender','age']

In [ ]:
inter_cols6 = []

for i in tqdm(bias_cols):
    
    for j in binned_cols :
        
        x_comb[i+'_'+j] = x_comb[i].astype(str) + '_' + x_comb[j].astype(str)
        
        inter_cols6.append(i+'_'+j)

In [ ]:
#Critical -3way interaction


x_comb['age'+'gender'+'weighted_classt'] = x_comb['age'].astype(str) + '_' + x_comb['gender'].astype(str) + '_' + x_comb['weightclass'].astype(str)
x_comb['age'+'gender'+'ethnicity'] = x_comb['age'].astype(str) + '_' + x_comb['gender'].astype(str) + '_' + x_comb['ethnicity'].astype(str)
x_comb['age'+'gender'+'ethnicity' +'weighted_classt' ] = x_comb['age'].astype(str) + '_' + x_comb['gender'].astype(str) + '_' + x_comb['ethnicity'].astype(str) +'_' + x_comb['weightclass'].astype(str)


inter_cols6 = inter_cols6 + ['age'+'gender'+'weighted_classt'] +['age'+'gender'+'ethnicity'] + ['age'+'gender'+'ethnicity'+'weighted_classt']

In [ ]:
for cols in tqdm(inter_cols6):
    le = LabelEncoder()
    x_comb[cols] = le.fit_transform(x_comb[cols].astype(str))

In [ ]:
list_illness = ['aids',
                'cirrhosis',
                'diabetes_mellitus',
                'hepatic_failure',
                'immunosuppression',
                'leukemia',
                'lymphoma',
                'solid_tumor_with_metastasis']


In [ ]:
inter_cols1 = []

for col in tqdm(list_illness) :
    x_comb['hospital_id'+str(col)] = x_comb['hospital_id'].astype(str) + '_' + x_comb[col].astype(str)
    x_comb['gender'+str(col)] = x_comb['gender'].astype(str) + '_' + x_comb[col].astype(str)
    x_comb['age'+str(col)] = x_comb['age'].astype(str) + '_' + x_comb[col].astype(str)
    x_comb['bmi'+str(col)] = x_comb['bmi'].astype(str) + '_' + x_comb[col].astype(str)
    
    inter_cols1.append('hospital_id'+str(col))
    inter_cols1.append('gender'+str(col))
    inter_cols1.append('age'+str(col))
    inter_cols1.append('bmi'+str(col))

In [ ]:
for cols in tqdm(inter_cols1):
    le = LabelEncoder()
    x_comb[cols] = le.fit_transform(x_comb[cols])
    

In [ ]:
inter_cols2 = []

for col in tqdm(list_illness) :
    x_comb['age_category'+str(col)] = x_comb['age_category'].astype(str) + '_' + x_comb[col].astype(str)
    inter_cols2.append('age_category'+str(col))

In [ ]:
for cols in tqdm(inter_cols2):
    le = LabelEncoder()
    x_comb[cols] = le.fit_transform(x_comb[cols])
    

In [ ]:
temp_train = x_train.copy()
temp_train['hospital_death'] = train_data['hospital_death']


In [ ]:
cat_count_var = []

for i,col in tqdm(enumerate(new_cat_var)):
    mean = temp_train['hospital_death'].mean()
    
    #Compute the number of values and the mean of each group
    agg = temp_train.groupby(col)['hospital_death'].agg(['count', 'mean'])
    counts = agg['count']
    means =  agg['mean']
    
    #Compute the "smoothed" means
    m= 3
    smooth = (counts * means + m * mean) / (counts + m)
    
    #Final_val
    x_comb[str(col)+'count_new'] = x_comb[col].map(smooth)
    cat_count_var.append(str(col)+'count_new')
    

In [ ]:
# aggs = {}
# aggs['apache_4a_hospital_death_prob'] = ['median', 'mean', 'max', 'min']
# aggs['apache_4a_icu_death_prob'] = ['median', 'mean', 'max', 'min']


In [ ]:
new_cat_var = list(set(new_cat_var).intersection(x_comb.columns))

In [ ]:
# for col in tqdm(new_cat_var):
    
#     aggs_temp = aggs.copy()
#     #aggs_temp.pop(col)
    
#     agg_df = x_comb.groupby(col).agg(aggs_temp).reset_index()
#     agg_df.columns = [col] + [col + '_' + c[0] +'_' + c[1] for c in agg_df.columns[1:]]
    
#     x_comb = pd.merge(left = x_comb, right= agg_df, how='left',
#                     left_on=[col], right_on=[col])
# cat_count_var2 = []

# for i,col in tqdm(enumerate(new_cat_var)):
#     mean = x_comb['apache_4a_hospital_death_prob'].mean()
    
#     #Compute the number of values and the mean of each group
#     agg = x_comb.groupby(col)['apache_4a_hospital_death_prob'].agg(['count', 'mean'])
#     counts = agg['count']
#     means =  agg['mean']
    
#     #Compute the "smoothed" means
#     m= 3
#     smooth = (counts * means + m * mean) / (counts + m)
    
#     #Final_val
#     x_comb[str(col)+'count_new'] = x_comb[col].map(smooth)
#     cat_count_var.append(str(col)+'count_new')
    

In [ ]:
# # Remove Features with more than 70 percent missing values
data_missing = (x_comb.isnull().sum() / len(x_comb)).sort_values(ascending = False)
data_missing = data_missing.index[data_missing > 0.9]

x_comb = x_comb.drop(columns = data_missing)

In [ ]:
cat_count_var_new = list(set(cat_count_var).intersection(x_comb.columns))
#x_comb.isna().sum()

In [ ]:
for col in tqdm(cat_count_var_new):
    le = LabelEncoder()
    x_comb[col] = le.fit_transform(x_comb[col].astype(str))

In [ ]:
def mode(x):
    return Counter(x).most_common(1)[0][0]


In [ ]:
aggs = {}

for col in tqdm(new_cat_var):
    aggs[col] = ['nunique','count',mode]
    

In [ ]:
new_cat_var = list(set(new_cat_var).intersection(set(x_comb.columns)))

In [ ]:
for col in tqdm(new_cat_var):
    
    aggs_temp = aggs.copy()
    #aggs_temp.pop(col)
    
    agg_df = x_comb.groupby(col).agg(aggs_temp).reset_index()
    agg_df.columns = [col] + [col + '_' + c[0] +'_' + c[1] for c in agg_df.columns[1:]]
    
    x_comb = pd.merge(left = x_comb, right= agg_df, how='left',
                    left_on=[col], right_on=[col])

In [ ]:
x_comb['hospital_icu_id'] = x_comb['hospital_id'].astype(str) + '_' +  x_comb['icu_id'].astype(str)

le = LabelEncoder()
x_comb['hospital_icu_id'] = le.fit_transform(x_comb['hospital_icu_id'].astype(str))

In [ ]:
#x_comb = x_comb.drop(['encounter_id','patient_id'],axis=1)

In [ ]:
#x_comb = x_comb.drop(list_unimp,axis=1)
x_comb.shape


In [ ]:
y_train.shape

In [ ]:
train_x = x_comb[0:91713]
test_x =  x_comb[91713:]

In [ ]:
x1_train, x1_val, y1_train, y1_val = train_test_split(train_x, y_train, 
                                                      test_size=0.2,
                                                      random_state=0,
                                                      stratify = y_train)

In [ ]:
random_state = 42
np.random.seed(random_state)

lgb_params = {
    'boost': 'gbdt',
    'learning_rate': 0.01,
    'max_depth':6,
    'metric':{'auc'},
    'num_threads': -1,
    'objective': 'binary',
    'verbosity': 1,
    'num_leaves':15,
    'bagging_fraction':0.5, #rows
    'bagging_frequency':5,
    'min_data_in_leaf':50,
    'feature_fraction':0.3, # features
    'lambda_l1':10,
    'lambda_l2':15,
    'bagging_seed':2019,
    'min_gain_split':0.3
}

In [ ]:
trn_data = lgb.Dataset(x1_train, label= y1_train)
val_data = lgb.Dataset(x1_val,  label=  y1_val)

In [ ]:
lgb_clf = lgb.train(lgb_params,
                    trn_data,
                    13000,
                    valid_sets = [trn_data, val_data],
                    early_stopping_rounds=1500,
                    verbose_eval=200
                    )

In [ ]:
predictions = lgb_clf.predict(test_x)

In [ ]:
data_sub = x_test[['encounter_id']]
data_sub['hospital_death'] = predictions

In [ ]:
data_sub.to_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/labs_apache_imputed_21thfeb_canbeoverfit.csv',index = False)


In [ ]:
feature_imp = pd.DataFrame(sorted(zip(lgb_clf.feature_importance(), 
                                      x_comb.columns), reverse=True), columns=['Value','Feature'])

In [ ]:
list_unimp = list(feature_imp[feature_imp['Value'] == 0]['Feature'])

In [ ]:
# fig, ax = plt.subplots(figsize=(12,30))
# lgb.plot_importance(lgb_clf, max_num_features=130, height=0.8, ax=ax)
# ax.grid(False)
# plt.title("LightGBM - Feature Importance", fontsize=12)
# plt.show()
# #data_sub

In [ ]:
kf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)

In [ ]:
predictions = np.zeros(len(x_test))
num_models = 0

for train_index, valid_index in tqdm(kf.split(train_x, y_train)):
    
    d_train = lgb.Dataset(train_x.iloc[train_index], label=y_train[train_index])
    d_val = lgb.Dataset(train_x.iloc[valid_index], label=y_train[valid_index])

    clf = lgb.train(lgb_params, d_train, 20000, verbose_eval=1000, 
                    valid_sets = [d_train, d_val], early_stopping_rounds = 1500)
    

    
    num_models += 1
    
    predictions += clf.predict(test_x)

In [ ]:
predictions_10 = predictions /10.0
data_sub = x_test[['encounter_id']]
data_sub['hospital_death'] = predictions_10

In [ ]:
data_sub.to_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/best_till_now_imputing_apache_20thfeb.csv',index = False)


In [ ]:
#feature_imp = pd.DataFrame(sorted(zip(lgb_clf.feature_importance(), 
                                      x_comb.columns), reverse=True), columns=['Value','Feature'])

In [ ]:
#feature_imp[feature_imp['Feature'] == 'weightclass']


In [ ]:
#feature_imp[feature_imp['Value'] == 0]

In [ ]:
a1 =  pd.read_csv('/Users/s0c02nj/Desktop/sub_10fold_keep_missing_as_it_is_18thFeb2020_newfeat.csv')
a2 =  pd.read_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/ensemble_till_now_12thfeb.csv')

In [ ]:
# a1 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/ensemble4_lgbold_10thfeb_new_bmi_impute.csv')
# a2 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_10fold_keep_missing_as_it_is_10thFeb2020_lgb_type4_ff_20_na_bmi.csv')
# #a2 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_10fold_keep_missing_as_it_is_9thFeb2020_lgb_type3_ff_27.csv')
# a3 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_type1_type2_ensemble_5thFeb2020.csv')
# a4 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_10fold_keep_missing_as_it_is_11thFeb2020_lgb_type3_ff_27.csv')
# a5 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/pred_lgb_keeping_cat_cont_interchanged_11th_feb.csv')
# a6 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/ensemble4tillnpw_11thfeb.csv')

In [ ]:
sub = pd.DataFrame()
sub['encounter_id'] = a1['encounter_id']

In [ ]:
pred_ens = (a1['hospital_death'] + a2['hospital_death']
           )/2.0
            

In [ ]:
sub['hospital_death'] = pred_ens

In [ ]:
sub.to_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/ensemble_till_now_18thfeb.csv',index = False)